# DataCamp Course Crawler

This notebook crawls course data from DataCamp, first collecting course names and URLs, then fetching detailed descriptions for each course.

In [14]:
# Install required packages
# !pip install selenium beautifulsoup4 tqdm

In [1]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from bs4 import BeautifulSoup
import time
import csv
import random
from tqdm import tqdm
from urllib.parse import urljoin
import pandas as pd

# Base URL
base_url = "https://www.datacamp.com/courses-all/page/{}"

# Set up Selenium WebDriver
def setup_driver():
    chrome_options = Options()
    # chrome_options.add_argument("--headless")  # Uncomment to run headlessly
    chrome_options.add_argument("--disable-gpu")
    chrome_options.add_argument("--no-sandbox")
    chrome_options.add_argument("--disable-dev-shm-usage")
    chrome_options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/130.0.0.0 Safari/537.36")
    
    # service = Service(port=9515)
    service = Service()
    driver = webdriver.Chrome(service=service, options=chrome_options)
    return driver

In [ ]:
# Step 1: Crawl course names and URLs
def crawl_course_links(page_num, csv_writer, driver):
    driver.execute_script("window.open('');")
    driver.switch_to.window(driver.window_handles[-1])
    
    url = base_url.format(page_num)
    print(f"\nCrawling page {page_num}: {url}")
    
    max_attempts = 3
    attempt = 1
    course_container = None
    
    while attempt <= max_attempts:
        try:
            driver.get(url)
            time.sleep(3)
            soup = BeautifulSoup(driver.page_source, 'html.parser')
            course_container = soup.find('section', class_='css-10jrqpr')
            if course_container:
                break
            else:
                print(f"Attempt {attempt}/{max_attempts}: No course container found on page {page_num}.")
                with open(f"debug_page_{page_num}.html", "w", encoding="utf-8") as f:
                    f.write(driver.page_source)
        except Exception as e:
            print(f"Attempt {attempt}/{max_attempts}: Failed to load page {page_num}: {e}")
        
        attempt += 1
        if attempt <= max_attempts:
            print(f"Retrying page {page_num} in 5-10 seconds...")
            time.sleep(random.uniform(5, 10))
    
    if not course_container:
        print(f"Failed to find course container on page {page_num} after {max_attempts} attempts.")
        driver.quit()
        driver = setup_driver()
        return False, driver

    course_links = course_container.find_all('a', class_='css-w9y587')
    if not course_links:
        print(f"No courses found on page {page_num}.")
        driver.quit()
        driver = setup_driver()
        return False, driver

    print(f"Found {len(course_links)} courses on page {page_num}")
    progress_bar = tqdm(course_links, desc=f"Page {page_num} Progress", unit="course")

    for course in progress_bar:
        course_name_tag = course.find('h2', class_='css-1k3iole')
        course_name = course_name_tag.get_text(strip=True) if course_name_tag else "Unknown Course"
        progress_bar.set_postfix({"Course": course_name[:30] + "..." if len(course_name) > 30 else course_name})
        
        course_url = course['href']
        if not course_url.startswith('http'):
            course_url = urljoin("https://www.datacamp.com", course_url)

        csv_writer.writerow([course_name, course_url, ""])
        print(f"\nCourse: {course_name}")
        print(f"URL: {course_url}")
        print("-" * 50)

        time.sleep(random.uniform(2, 5))

    progress_bar.close()
    driver.quit()
    driver = setup_driver()
    return True, driver

In [12]:
def crawl_descriptions():
    df = pd.read_csv("datacamp_courses.csv")
    print(f"Loaded {len(df)} courses from CSV.")

    # File mới sẽ chứa kết quả đầy đủ
    output_file = "datacamp_courses_full.csv"

    # Ghi tiêu đề nếu file chưa tồn tại
    with open(output_file, 'w', encoding='utf-8', newline='') as f:
        writer = csv.writer(f)
        writer.writerow(["Course Name", "Course URL", "Detailed Description"])

    progress_bar = tqdm(range(len(df)), desc="Fetching Descriptions", unit="course")

    for idx in progress_bar:
        course_name = df.iloc[idx]["Course Name"]
        course_url = df.iloc[idx]["Course URL"]
        progress_bar.set_postfix({"Course": course_name[:30] + "..." if len(course_name) > 30 else course_name})

        try:
            driver = setup_driver()  # mở trình duyệt mới
            driver.get(course_url)
            time.sleep(random.uniform(4, 6))

            soup = BeautifulSoup(driver.page_source, "html.parser")
            description_section = soup.find("article", class_="css-evox4w")
            description_text = description_section.get_text(strip=True, separator=" | ") if description_section else "No detailed description available"
            description_text = description_text.replace("\n", " ").replace("\r", " ").strip()  # ← thêm dòng này


            # ✅ Ghi vào file mới
            with open(output_file, 'a', encoding='utf-8', newline='') as f:
                writer = csv.writer(f)
                writer.writerow([course_name, course_url, description_text])

            # ✅ In ra log
            print(f"\n[✔] {course_name}:\n{description_text[:200]}{'...' if len(description_text) > 200 else ''}\n")

        except Exception as e:
            print(f"❌ Error for {course_name}: {e}")
            with open(output_file, 'a', encoding='utf-8', newline='') as f:
                writer = csv.writer(f)
                writer.writerow([course_name, course_url, "Error fetching description"])

        finally:
            driver.quit()
            time.sleep(random.uniform(3, 6))

    progress_bar.close()
    print(f"\n🎉 Full data written to {output_file}")


In [4]:
# Main execution
output_file = "datacamp_courses.csv"
max_pages = 20

# Initialize CSV
with open(output_file, 'w', encoding='utf-8', newline='') as f:
    csv_writer = csv.writer(f)
    csv_writer.writerow(["Course Name", "Course URL", "Detailed Description"])

driver = setup_driver()
try:
    page_num = 1
    while page_num <= max_pages:
        with open(output_file, 'a', encoding='utf-8', newline='') as f:
            csv_writer = csv.writer(f)
            success, driver = crawl_course_links(page_num, csv_writer, driver)  # nhận lại driver
        if not success:
            print(f"Finished crawling after page {page_num}")
            break
        page_num += 1
        time.sleep(random.uniform(5, 10))

finally:
    driver.quit()

print(f"\nCrawling complete. Results saved to {output_file}")


Crawling page 1: https://www.datacamp.com/courses-all/page/1
Found 30 courses on page 1


Page 1 Progress:   0%|          | 0/30 [00:00<?, ?course/s, Course=Introduction to Python]


Course: Introduction to Python
URL: https://www.datacamp.com/courses/intro-to-python-for-data-science
--------------------------------------------------


Page 1 Progress:   3%|▎         | 1/30 [00:02<01:09,  2.38s/course, Course=Introduction to SQL]   


Course: Introduction to SQL
URL: https://www.datacamp.com/courses/introduction-to-sql
--------------------------------------------------


...


Course: Scalable AI Models with PyTorch Lightning
URL: https://www.datacamp.com/courses/scalable-ai-models-with-pytorch-lightning
--------------------------------------------------


Page 20 Progress:  50%|█████     | 1/2 [00:04<00:04,  4.75s/course, Course=Introduction to Sigma]            


Course: Introduction to Sigma
URL: https://www.datacamp.com/courses/introduction-to-sigma
--------------------------------------------------


Page 20 Progress: 100%|██████████| 2/2 [00:07<00:00,  3.52s/course, Course=Introduction to Sigma]



Crawling complete. Results saved to datacamp_courses.csv


In [13]:
driver = setup_driver()
try:
    # Step 2: Crawl descriptions
    crawl_descriptions()
    
finally:
    driver.quit()
    
print("Crawling descriptions complete!")

Loaded 572 courses from CSV.


Fetching Descriptions:   0%|          | 0/572 [00:00<?, ?course/s, Course=Introduction to Python]


[✔] Introduction to Python:
An Introduction to Python | Python has grown to become the market leader in programming languages and the language of choice for data analysts and data scientists. Demand for data skills is rising bec...



Fetching Descriptions:   0%|          | 1/572 [00:20<3:16:02, 20.60s/course, Course=Introduction to SQL]   


[✔] Introduction to SQL:
Get an Introduction to SQL in Two Hours | Much of the world's raw data—from electronic medical records to customer transaction histories—lives in organized collections of tables called relational data...



...


[✔] Scalable AI Models with PyTorch Lightning:
Foundations of Scalable AI | This course takes you on a journey through the fundamentals of scalable AI. You’ll begin by learning how PyTorch Lightning streamlines the model development lifecycle by r...



Fetching Descriptions: 100%|█████████▉| 571/572 [3:24:56<00:18, 18.96s/course, Course=Introduction to Sigma]            


[✔] Introduction to Sigma:
This introduction to Sigma course helps data-curious professionals quickly get up to speed with Sigma’s powerful, spreadsheet-like interface.  Whether you’re new to data analytics or transitioning fro...



Fetching Descriptions: 100%|██████████| 572/572 [3:25:14<00:00, 21.53s/course, Course=Introduction to Sigma]



🎉 Full data written to datacamp_courses_full.csv
Crawling descriptions complete!
